#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter Twitter Targeting Parameters
Adjusts line item settings based on Twitter hashtags and locations specified in a sheet.
 1. Provide a sheets URL. No account information required.
 1. Click <b>Run Now</b> once and a new tab called <b>Twitter Triggers</b> will be added to the sheet.
 1. Follow instructions on the sheets tab.
 1. Click <b>Run Now</b> again, trends are downloaded and triggered
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  "dataset":"", # Place where tables will be created in BigQuery.
  "sheet_url":"", # URL to sheet where Line Item settings will be read from.
  "twitter_secret":"", # Twitter API secret token.
  "twitter_key":"", # Twitter API key token.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute Twitter Targeting
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [ { 'dataset': { 'auth': 'user',
                 'dataset': { 'field': { 'description': 'Place '
                                                        'where '
                                                        'tables '
                                                        'will '
                                                        'be '
                                                        'created '
                                                        'in '
                                                        'BigQuery.',
                                         'kind': 'string',
                                         'name': 'dataset',
                                         'order': 1}},
                 'description': 'Create '
                                'a '
                                'dataset '
                                'where '
                                'data '
                                'will '
                                'be '
                                'combined '
                                'and '
                                'transfored '
                                'for '
                                'upload.'}},
  { 'sheets': { 'auth': 'user',
                'description': 'Read '
                               'mapping '
                               'of '
                               'hash '
                               'tags '
                               'to '
                               'line '
                               'item '
                               'toggles '
                               'from '
                               'sheets.',
                'out': { 'bigquery': { 'dataset': { 'field': { 'description': 'Place '
                                                                              'where '
                                                                              'tables '
                                                                              'will '
                                                                              'be '
                                                                              'created '
                                                                              'in '
                                                                              'BigQuery.',
                                                               'kind': 'string',
                                                               'name': 'dataset'}},
                                       'schema': [ { 'mode': 'REQUIRED',
                                                     'name': 'Location',
                                                     'type': 'STRING'},
                                                   { 'mode': 'REQUIRED',
                                                     'name': 'WOEID',
                                                     'type': 'INTEGER'},
                                                   { 'mode': 'REQUIRED',
                                                     'name': 'Hashtag',
                                                     'type': 'STRING'},
                                                   { 'mode': 'REQUIRED',
                                                     'name': 'Line_Item_Id',
                                                     'type': 'INTEGER'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Name',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Status',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Start_Date',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_End_Date',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Budget_Type',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Budget_Amount',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Pacing',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Pacing_Rate',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Pacing_Amount',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Frequency_Enabled',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Frequency_Exposures',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Frequency_Period',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Line_Item_Frequency_Amount',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Bid_Price',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Partner_Revenue_Model',
                                                     'type': 'STRING'},
                                                   { 'mode': 'NULLABLE',
                                                     'name': 'Partner_Revenue_Amount',
                                                     'type': 'STRING'}],
                                       'table': 'Twitter_Triggers'}},
                'range': 'A8:T',
                'sheet': { 'field': { 'default': '',
                                      'description': 'URL '
                                                     'to '
                                                     'sheet '
                                                     'where '
                                                     'Line '
                                                     'Item '
                                                     'settings '
                                                     'will '
                                                     'be '
                                                     'read '
                                                     'from.',
                                      'kind': 'string',
                                      'name': 'sheet_url',
                                      'order': 2}},
                'tab': 'Twitter '
                       'Triggers',
                'template': { 'tab': 'Twitter '
                                     'Triggers',
                              'url': 'https://docs.google.com/spreadsheets/d/1iYCGa2NKOZiL2mdT4yiDfV_SWV9C7SUosXdIr4NAEXE/edit?usp=sharing'}}},
  { 'twitter': { 'auth': 'user',
                 'description': 'Read '
                                'trends '
                                'from '
                                'Twitter '
                                'and '
                                'place '
                                'into '
                                'BigQuery.',
                 'key': { 'field': { 'default': '',
                                     'description': 'Twitter '
                                                    'API '
                                                    'key '
                                                    'token.',
                                     'kind': 'string',
                                     'name': 'twitter_key',
                                     'order': 4}},
                 'out': { 'bigquery': { 'dataset': { 'field': { 'description': 'Place '
                                                                               'where '
                                                                               'tables '
                                                                               'will '
                                                                               'be '
                                                                               'created '
                                                                               'in '
                                                                               'BigQuery.',
                                                                'kind': 'string',
                                                                'name': 'dataset'}},
                                        'table': 'Twitter_Trends_Place'}},
                 'secret': { 'field': { 'default': '',
                                        'description': 'Twitter '
                                                       'API '
                                                       'secret '
                                                       'token.',
                                        'kind': 'string',
                                        'name': 'twitter_secret',
                                        'order': 3}},
                 'trends': { 'places': { 'bigquery': { 'columns': [ 'WOEID'],
                                                       'dataset': { 'field': { 'description': 'Place '
                                                                                              'where '
                                                                                              'tables '
                                                                                              'will '
                                                                                              'be '
                                                                                              'created '
                                                                                              'in '
                                                                                              'BigQuery.',
                                                                               'kind': 'string',
                                                                               'name': 'dataset'}},
                                                       'table': 'Twitter_Triggers'}}}}},
  { 'lineitem': { 'auth': 'user',
                  'description': 'Read '
                                 'current '
                                 'lineitem '
                                 'settings '
                                 'from '
                                 'DBM '
                                 'into '
                                 'BigQuery, '
                                 'so '
                                 'it '
                                 'can '
                                 'be '
                                 'joined '
                                 'with '
                                 'Twitter '
                                 'analysis.',
                  'read': { 'advertisers': [ ],
                            'insertionorders': [ ],
                            'lineitems': [ 9915840],
                            'out': { 'bigquery': { 'dataset': { 'field': { 'description': 'Place '
                                                                                          'where '
                                                                                          'tables '
                                                                                          'will '
                                                                                          'be '
                                                                                          'created '
                                                                                          'in '
                                                                                          'BigQuery.',
                                                                           'kind': 'string',
                                                                           'name': 'dataset'}},
                                                   'table': 'LineItem_Reads'}}}}},
  { 'bigquery': { 'auth': 'user',
                  'description': 'Get '
                                 'all '
                                 'triggered '
                                 'lineitmes '
                                 'from '
                                 'sheet, '
                                 'if '
                                 'they '
                                 'have '
                                 'a '
                                 'keyword '
                                 'match '
                                 'in '
                                 'twitter, '
                                 'take '
                                 'the '
                                 'triger '
                                 'values, '
                                 'else '
                                 'take '
                                 'the '
                                 'default '
                                 'values '
                                 '(default>trigger).  '
                                 'Take '
                                 'all '
                                 'non-null '
                                 'values '
                                 'from '
                                 'trigger '
                                 'and '
                                 'overlay '
                                 'over '
                                 'current '
                                 'DBM '
                                 'values. '
                                 'Will '
                                 'be '
                                 'used '
                                 'to '
                                 'upload '
                                 'to '
                                 'DBM.',
                  'from': { 'legacy': False,
                            'parameters': [ { 'field': { 'description': 'Place '
                                                                        'where '
                                                                        'tables '
                                                                        'will '
                                                                        'be '
                                                                        'created '
                                                                        'in '
                                                                        'BigQuery.',
                                                         'kind': 'string',
                                                         'name': 'dataset'}},
                                            { 'field': { 'description': 'Place '
                                                                        'where '
                                                                        'tables '
                                                                        'will '
                                                                        'be '
                                                                        'created '
                                                                        'in '
                                                                        'BigQuery.',
                                                         'kind': 'string',
                                                         'name': 'dataset'}},
                                            { 'field': { 'description': 'Place '
                                                                        'where '
                                                                        'tables '
                                                                        'will '
                                                                        'be '
                                                                        'created '
                                                                        'in '
                                                                        'BigQuery.',
                                                         'kind': 'string',
                                                         'name': 'dataset'}}],
                            'query': 'SELECT '
                                     'o.Line_Item_Id '
                                     'AS '
                                     'Line_Item_Id, '
                                     'o.Partner_Name '
                                     'AS '
                                     'Partner_Name, '
                                     'o.Partner_Id '
                                     'AS '
                                     'Partner_Id, '
                                     'o.Advertiser_Name '
                                     'AS '
                                     'Advertiser_Name, '
                                     'o.IO_Name '
                                     'AS '
                                     'IO_Name, '
                                     'IFNULL(t.Line_Item_Name, '
                                     'o.Line_Item_Name) '
                                     'AS '
                                     'Line_Item_Name, '
                                     'o.Line_Item_Timestamp '
                                     'AS '
                                     'Line_Item_Timestamp '
                                     ', '
                                     'IFNULL(t.Line_Item_Status, '
                                     'o.Line_Item_Status) '
                                     'AS '
                                     'Line_Item_Status, '
                                     'o.IO_Start_Date '
                                     'AS '
                                     'IO_Start_Date, '
                                     'o.IO_End_Date '
                                     'AS '
                                     'IO_End_Date, '
                                     'o.IO_Budget_Type '
                                     'AS '
                                     'IO_Budget_Type, '
                                     'o.IO_Budget_Amount '
                                     'AS '
                                     'IO_Budget_Amount, '
                                     'o.IO_Pacing '
                                     'AS '
                                     'IO_Pacing, '
                                     'o.IO_Pacing_Rate '
                                     'AS '
                                     'IO_Pacing_Rate, '
                                     'o.IO_Pacing_Amount '
                                     'AS '
                                     'IO_Pacing_Amount, '
                                     'IFNULL(t.Line_Item_Start_Date, '
                                     'o.Line_Item_Start_Date) '
                                     'AS '
                                     'Line_Item_Start_Date, '
                                     'IFNULL(t.Line_Item_End_Date, '
                                     'o.Line_Item_End_Date) '
                                     'AS '
                                     'Line_Item_End_Date, '
                                     'IFNULL(t.Line_Item_Budget_Type, '
                                     'o.Line_Item_Budget_Type) '
                                     'AS '
                                     'Line_Item_Budget_Type, '
                                     'IFNULL(t.Line_Item_Budget_Amount, '
                                     'o.Line_Item_Budget_Amount) '
                                     'AS '
                                     'Line_Item_Budget_Amount, '
                                     'IFNULL(t.Line_Item_Pacing, '
                                     'o.Line_Item_Pacing) '
                                     'AS '
                                     'Line_Item_Pacing, '
                                     'IFNULL(t.Line_Item_Pacing_Rate, '
                                     'o.Line_Item_Pacing_Rate) '
                                     'AS '
                                     'Line_Item_Pacing_Rate, '
                                     'IFNULL(t.Line_Item_Pacing_Amount, '
                                     'o.Line_Item_Pacing_Amount) '
                                     'AS '
                                     'Line_Item_Pacing_Amount, '
                                     'IFNULL(t.Line_Item_Frequency_Enabled, '
                                     'o.Line_Item_Frequency_Enabled) '
                                     'AS '
                                     'Line_Item_Frequency_Enabled, '
                                     'IFNULL(t.Line_Item_Frequency_Exposures, '
                                     'o.Line_Item_Frequency_Exposures) '
                                     'AS '
                                     'Line_Item_Frequency_Exposures, '
                                     'IFNULL(t.Line_Item_Frequency_Period, '
                                     'o.Line_Item_Frequency_Period) '
                                     'AS '
                                     'Line_Item_Frequency_Period, '
                                     'IFNULL(t.Line_Item_Frequency_Amount, '
                                     'o.Line_Item_Frequency_Amount) '
                                     'AS '
                                     'Line_Item_Frequency_Amount, '
                                     'IFNULL(t.Bid_Price, '
                                     'o.Bid_Price) '
                                     'AS '
                                     'Bid_Price, '
                                     'IFNULL(t.Partner_Revenue_Model, '
                                     'o.Partner_Revenue_Model) '
                                     'AS '
                                     'Partner_Revenue_Model, '
                                     'IFNULL(t.Partner_Revenue_Amount, '
                                     'o.Partner_Revenue_Amount) '
                                     'AS '
                                     'Partner_Revenue_Amount, '
                                     'o.Current_Audience_Targeting_Ids '
                                     'AS '
                                     'Current_Audience_Targeting_Ids '
                                     ', '
                                     'o.Current_Audience_Targeting_Names '
                                     'AS '
                                     'Current_Audience_Targeting_Names '
                                     'FROM '
                                     '`[PARAMETER].LineItem_Reads` '
                                     'AS '
                                     'o '
                                     'LEFT '
                                     'JOIN '
                                     '( '
                                     'SELECT '
                                     'Line_Item_Id, '
                                     'ANY_VALUE(SPLIT(Line_Item_Name, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Line_Item_Name, '
                                     'ANY_VALUE(SPLIT(Line_Item_Status, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Line_Item_Status, '
                                     'ANY_VALUE(SPLIT(Line_Item_Start_Date, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Line_Item_Start_Date, '
                                     'ANY_VALUE(SPLIT(Line_Item_End_Date, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Line_Item_End_Date, '
                                     'ANY_VALUE(SPLIT(Line_Item_Budget_Type, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Line_Item_Budget_Type, '
                                     'ANY_VALUE(CAST(SPLIT(Line_Item_Budget_Amount, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))] '
                                     'AS '
                                     'FLOAT64)) '
                                     'AS '
                                     'Line_Item_Budget_Amount, '
                                     'ANY_VALUE(SPLIT(Line_Item_Pacing, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Line_Item_Pacing, '
                                     'ANY_VALUE(SPLIT(Line_Item_Pacing_Rate, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Line_Item_Pacing_Rate, '
                                     'ANY_VALUE(CAST(SPLIT(Line_Item_Pacing_Amount, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))] '
                                     'AS '
                                     'FLOAT64)) '
                                     'AS '
                                     'Line_Item_Pacing_Amount, '
                                     'ANY_VALUE(CAST(SPLIT(Line_Item_Frequency_Enabled, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))] '
                                     'AS '
                                     'BOOL)) '
                                     'AS '
                                     'Line_Item_Frequency_Enabled, '
                                     'ANY_VALUE(SPLIT(Line_Item_Frequency_Exposures, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Line_Item_Frequency_Exposures, '
                                     'ANY_VALUE(SPLIT(Line_Item_Frequency_Period, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Line_Item_Frequency_Period, '
                                     'ANY_VALUE(CAST(SPLIT(Line_Item_Frequency_Amount, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))] '
                                     'AS '
                                     'INT64)) '
                                     'AS '
                                     'Line_Item_Frequency_Amount, '
                                     'ANY_VALUE(CAST(SPLIT(Bid_Price, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))] '
                                     'AS '
                                     'FLOAT64)) '
                                     'AS '
                                     'Bid_Price, '
                                     'ANY_VALUE(SPLIT(Partner_Revenue_Model, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))]) '
                                     'AS '
                                     'Partner_Revenue_Model, '
                                     'ANY_VALUE(CAST(SPLIT(Partner_Revenue_Amount, '
                                     "'>')[SAFE_OFFSET(IF(Triggered, "
                                     '1, '
                                     '0))] '
                                     'AS '
                                     'FLOAT64)) '
                                     'AS '
                                     'Partner_Revenue_Amount '
                                     'FROM '
                                     '( '
                                     'SELECT '
                                     'WOEID, '
                                     'Hashtag, '
                                     'Line_Item_Id, '
                                     'Line_Item_Name, '
                                     'Line_Item_Status, '
                                     'Line_Item_Start_Date, '
                                     'Line_Item_End_Date, '
                                     'Line_Item_Budget_Type, '
                                     'Line_Item_Budget_Amount, '
                                     'Line_Item_Pacing, '
                                     'Line_Item_Pacing_Rate, '
                                     'Line_Item_Pacing_Amount, '
                                     'Line_Item_Frequency_Enabled, '
                                     'Line_Item_Frequency_Exposures, '
                                     'Line_Item_Frequency_Period, '
                                     'Line_Item_Frequency_Amount, '
                                     'Bid_Price, '
                                     'Partner_Revenue_Model, '
                                     'Partner_Revenue_Amount, '
                                     'CONCAT(CAST(WOEID '
                                     'AS '
                                     'STRING), '
                                     'LOWER(Hashtag)) '
                                     'IN '
                                     '(SELECT '
                                     'CONCAT(CAST(WOEID '
                                     'AS '
                                     'STRING), '
                                     'LOWER(REPLACE(name, '
                                     "'#', "
                                     "''))) "
                                     'FROM '
                                     '`[PARAMETER].Twitter_Trends_Place` '
                                     'GROUP '
                                     'BY '
                                     '1) '
                                     'AS '
                                     'Triggered '
                                     'FROM '
                                     '`[PARAMETER].Twitter_Triggers`) '
                                     'GROUP '
                                     'BY '
                                     '1) '
                                     'AS '
                                     't '
                                     'ON '
                                     'o.Line_Item_Id=t.Line_Item_Id;'},
                  'to': { 'dataset': { 'field': { 'description': 'Place '
                                                                 'where '
                                                                 'tables '
                                                                 'will '
                                                                 'be '
                                                                 'created '
                                                                 'in '
                                                                 'BigQuery.',
                                                  'kind': 'string',
                                                  'name': 'dataset'}},
                          'view': 'LineItem_Writes'}}},
  { 'lineitem': { 'auth': 'user',
                  'description': 'Write '
                                 'lineitem '
                                 'settings '
                                 'to '
                                 'DBM '
                                 'after '
                                 'transformation.',
                  'write': { 'bigquery': { 'dataset': { 'field': { 'description': 'Place '
                                                                                  'where '
                                                                                  'tables '
                                                                                  'will '
                                                                                  'be '
                                                                                  'created '
                                                                                  'in '
                                                                                  'BigQuery.',
                                                                   'kind': 'string',
                                                                   'name': 'dataset'}},
                                           'table': 'LineItem_Writes'},
                             'dry_run': False}}}]

json_set_fields(TASKS, FIELDS)
project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True)
project.execute()
